## SRCNN

We first implement a basic SRCNN model using tensorflow.
References: 

In [11]:
import os
import glob
import h5py
import random
import matplotlib.pyplot as plt

from PIL import Image
import scipy.misc
import scipy.ndimage
import numpy as np
import imageio

import tensorflow as tf


def read_data(path):
    """
    Read h5 format data file
    
    Args:
        path: file path of desired file
        data: '.h5' file format that contains train data values
        label: '.h5' file format that contains train label values
    """
    with h5py.File(path, 'r') as hf:
        data = np.array(hf.get('data'))
        label = np.array(hf.get('label'))
        return data, label

def preprocess(path, scale=3):
    """
    Preprocess single image file 
        (1) Read original image as YCbCr format (and grayscale as default)
        (2) Normalize
        (3) Apply image file with bicubic interpolation
    Args:
        path: file path of desired file
        input_: image applied bicubic interpolation (low-resolution)
        label_: image with original resolution (high-resolution)
    """
    image = imread(path, is_grayscale=True)
    label_ = modcrop(image, scale)

    # Must be normalized
    image = image / 255.
    label_ = label_ / 255.

    input_ = scipy.ndimage.interpolation.zoom(label_, (1./scale), prefilter=False)
    input_ = scipy.ndimage.interpolation.zoom(input_, (scale/1.), prefilter=False)

    return input_, label_

def prepare_data(sess, dataset, is_train):
    if is_train:
        filenames = os.listdir(dataset)
        data_dir = os.path.join(os.getcwd(), dataset)
        data = glob.glob(os.path.join(data_dir, "*.bmp"))
    else:
        data_dir = os.path.join(os.sep, (os.path.join(os.getcwd(), dataset)), "Set5")
        data = glob.glob(os.path.join(data_dir, "*.bmp"))

    return data

def make_data(sess, data, label, is_train):
    if is_train:
        savepath = os.path.join(os.getcwd(), 'checkpoint/train.h5')
    else:
        savepath = os.path.join(os.getcwd(), 'checkpoint/test.h5')

    with h5py.File(savepath, 'w') as hf:
        hf.create_dataset('data', data=data)
        hf.create_dataset('label', data=label)

def rgb2ycbcr(im):
    xform = np.array([[.299, .587, .114], [-.1687, -.3313, .5], [.5, -.4187, -.0813]])
    ycbcr = im.dot(xform.T)
    ycbcr[:,:,[1,2]] += 128
    return np.uint8(ycbcr)

def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

def imread(path, is_grayscale=True):
    if is_grayscale:
        return rgb2gray(rgb2ycbcr(imageio.imread(path).astype(np.float)))
    else:
        return rgb2ycbcr(imageio.imread(path).astype(np.float))

def modcrop(image, scale=3):
    if len(image.shape) == 3:
        h, w, _ = image.shape
        h = h - np.mod(h, scale)
        w = w - np.mod(w, scale)
        image = image[0:h, 0:w, :]
    else:
        h, w = image.shape
        h = h - np.mod(h, scale)
        w = w - np.mod(w, scale)
        image = image[0:h, 0:w]
    return image

def input_setup(sess, config):
    # Load data path
    if config.is_train:
        data = prepare_data(sess, dataset="Train", is_train=True)
    else:
        data = prepare_data(sess, dataset="Test", is_train=False)

    sub_input_sequence = []
    sub_label_sequence = []
    padding = abs(config.image_size - config.label_size) / 2 # 6

    if config.is_train:
        for i in range(len(data)):
            input_, label_ = preprocess(data[i], config.scale)

            if len(input_.shape) == 3:
                h, w, _ = input_.shape
            else:
                h, w = input_.shape

            for x in range(0, h-config.image_size+1, config.stride):
                for y in range(0, w-config.image_size+1, config.stride):
                    sub_input = input_[x:x+config.image_size, y:y+config.image_size] # [33 x 33]
                    sub_label = label_[x+int(padding):x+int(padding)+config.label_size, y+int(padding):y+int(padding)+config.label_size] # [21 x 21]

                    # Make channel value
                    sub_input = sub_input.reshape([config.image_size, config.image_size, 1])  
                    sub_label = sub_label.reshape([config.label_size, config.label_size, 1])

                    sub_input_sequence.append(sub_input)
                    sub_label_sequence.append(sub_label)

    else:
        input_, label_ = preprocess(data[2], config.scale)

        if len(input_.shape) == 3:
            h, w, _ = input_.shape
        else:
            h, w = input_.shape

        # Numbers of sub-images in height and width of image are needed to compute merge operation.
        nx = ny = 0 
        for x in range(0, h-config.image_size+1, config.stride):
            nx += 1; ny = 0
            for y in range(0, w-config.image_size+1, config.stride):
                ny += 1
                sub_input = input_[x:x+config.image_size, y:y+config.image_size] # [33 x 33]
                sub_label = label_[x+int(padding):x+int(padding)+config.label_size, y+int(padding):y+int(padding)+config.label_size] # [21 x 21]
                
                sub_input = sub_input.reshape([config.image_size, config.image_size, 1])  
                sub_label = sub_label.reshape([config.label_size, config.label_size, 1])

                sub_input_sequence.append(sub_input)
                sub_label_sequence.append(sub_label)

    """
    len(sub_input_sequence) : the number of sub_input (33 x 33 x ch) in one image
    (sub_input_sequence[0]).shape : (33, 33, 1)
    """
    # Make list to numpy array. With this transform
    arrdata = np.asarray(sub_input_sequence) # [?, 33, 33, 1]
    arrlabel = np.asarray(sub_label_sequence) # [?, 21, 21, 1]

    make_data(sess, arrdata, arrlabel, config.is_train)

    if not config.is_train:
        return nx, ny
        
def imsave(image, path):
    return scipy.misc.imsave(path, image)

def merge(images, size):
    h, w = images.shape[1], images.shape[2]
    img = np.zeros((h*size[0], w*size[1], 1))
    for idx, image in enumerate(images):
        i = idx % size[1]
        j = idx // size[1]
        img[j*h:j*h+h, i*w:i*w+w, :] = image

    return img

In [12]:
import time
import os
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf

try:
    xrange
except:
    xrange = range

class SRCNN(object):

    def __init__(self, sess, image_size=33,label_size=21, batch_size=128,c_dim=1, checkpoint_dir=None, sample_dir=None, is_train=True):

        self.sess = sess
        self.is_grayscale = (c_dim == 1)
        self.image_size = image_size
        self.label_size = label_size
        self.batch_size = batch_size

        self.c_dim = c_dim

        self.checkpoint_dir = checkpoint_dir
        self.sample_dir = sample_dir
        self.is_train = is_train
        self.build_model()

    def build_model(self):
        self.images = tf.placeholder(tf.float32, [None, self.image_size, self.image_size, self.c_dim], name='images')
        self.labels = tf.placeholder(tf.float32, [None, self.label_size, self.label_size, self.c_dim], name='labels')
        
        self.weights = {
            'w1': tf.Variable(tf.random_normal([9, 9, 1, 64], stddev=1e-3), name='w1'),
            'w2': tf.Variable(tf.random_normal([1, 1, 64, 32], stddev=1e-3), name='w2'),
            'w3': tf.Variable(tf.random_normal([5, 5, 32, 1], stddev=1e-3), name='w3')
        }
        self.biases = {
            'b1': tf.Variable(tf.zeros([64]), name='b1'),
            'b2': tf.Variable(tf.zeros([32]), name='b2'),
            'b3': tf.Variable(tf.zeros([1]), name='b3')
        }

        self.pred = self.model()

        # Loss function (MSE)
        self.loss = tf.reduce_mean(tf.square(self.labels - self.pred))

        self.saver = tf.train.Saver()

    def train(self, config):
        if config.is_train:
            input_setup(self.sess, config)
        else:
            nx, ny = input_setup(self.sess, config)

        if config.is_train:     
            data_dir = os.path.join('./{}'.format(config.checkpoint_dir), "train.h5")
        else:
            data_dir = os.path.join('./{}'.format(config.checkpoint_dir), "test.h5")

        train_data, train_label = read_data(data_dir)

        # Stochastic gradient descent with the standard backpropagation
        self.train_op = tf.train.GradientDescentOptimizer(config.learning_rate).minimize(self.loss)

        tf.initialize_all_variables().run()
        
        counter = 0
        start_time = time.time()

        if self.load(self.checkpoint_dir):
            print(" [*] Load SUCCESS")
        else:
            print(" [!] Load failed...")

        if config.is_train:
            print("Training...")

            for ep in xrange(config.epoch):
                # Run by batch images
                batch_idxs = len(train_data) // config.batch_size
                for idx in xrange(0, batch_idxs):
                    batch_images = train_data[idx*config.batch_size : (idx+1)*config.batch_size]
                    batch_labels = train_label[idx*config.batch_size : (idx+1)*config.batch_size]

                    counter += 1
                    _, err = self.sess.run([self.train_op, self.loss], feed_dict={self.images: batch_images, self.labels: batch_labels})

                    if counter % 10 == 0:
                        print("Epoch: [%2d], step: [%2d], time: [%4.4f], loss: [%.8f]" \
                            % ((ep+1), counter, time.time()-start_time, err))

                    if counter % 500 == 0:
                        self.save(config.checkpoint_dir, counter)

        else:
            print("Testing...")

            result = self.pred.eval({self.images: train_data, self.labels: train_label})

            result = merge(result, [nx, ny])
            result = result.squeeze()
            image_path = os.path.join(os.getcwd(), config.sample_dir)
            image_path = os.path.join(image_path, "test_image.png")
            imsave(result, image_path)

    def model(self):
        conv1 = tf.nn.relu(tf.nn.conv2d(self.images, self.weights['w1'], strides=[1,1,1,1], padding='VALID') + self.biases['b1'])
        conv2 = tf.nn.relu(tf.nn.conv2d(conv1, self.weights['w2'], strides=[1,1,1,1], padding='VALID') + self.biases['b2'])
        conv3 = tf.nn.conv2d(conv2, self.weights['w3'], strides=[1,1,1,1], padding='VALID') + self.biases['b3']
        return conv3

    def save(self, checkpoint_dir, step):
        model_name = "SRCNN.model"
        model_dir = "%s_%s" % ("srcnn", self.label_size)
        checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        if not os.path.exists(checkpoint_dir):
                os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,
                                        os.path.join(checkpoint_dir, model_name),
                                        global_step=step)

    def load(self, checkpoint_dir):
        print(" [*] Reading checkpoints...")
        model_dir = "%s_%s" % ("srcnn", self.label_size)
        checkpoint_dir = os.path.join(checkpoint_dir, model_dir)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
                ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
                self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
                return True
        else:
                return False

In [13]:
import numpy as np
import tensorflow as tf

import os

class config_c:
    def __init__(self):
        self.epoch=15000
        self.batch_size=128
        self.image_size=33
        self.label_size=21
        self.learning_rate=1e-4
        self.c_dim=1
        self.scale=3
        self.stride=14
        self.checkpoint_dir="checkpoint"
        self.sample_dir="sample"
        self.is_train=True

config=config_c()

if not os.path.exists(config.checkpoint_dir):
    os.makedirs(config.checkpoint_dir)
if not os.path.exists(config.sample_dir):
    os.makedirs(config.sample_dir)

with tf.Session() as sess:
    srcnn = SRCNN(sess,image_size=33,label_size=21,batch_size=128,c_dim=1,checkpoint_dir="checkpoint",sample_dir="sample",is_train=True)

    srcnn.train(config)
    tf.app.run()

/home/humanoid/o-conda/anaconda3/envs/tf1.x/lib/python3.7/site-packages/ipykernel_launcher.py:85: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations



Instructions for updating:
Use `tf.global_variables_initializer` instead.
 [*] Reading checkpoints...
 [!] Load failed...
Training...


OMP: Info #254: KMP_AFFINITY: pid 3649226 tid 3654859 thread 1 bound to OS proc set 1
OMP: Info #254: KMP_AFFINITY: pid 3649226 tid 3655481 thread 2 bound to OS proc set 2
OMP: Info #254: KMP_AFFINITY: pid 3649226 tid 3655482 thread 3 bound to OS proc set 3
OMP: Info #254: KMP_AFFINITY: pid 3649226 tid 3655483 thread 4 bound to OS proc set 4
OMP: Info #254: KMP_AFFINITY: pid 3649226 tid 3655484 thread 5 bound to OS proc set 5
OMP: Info #254: KMP_AFFINITY: pid 3649226 tid 3655485 thread 6 bound to OS proc set 6
OMP: Info #254: KMP_AFFINITY: pid 3649226 tid 3655486 thread 7 bound to OS proc set 7
OMP: Info #254: KMP_AFFINITY: pid 3649226 tid 3655487 thread 8 bound to OS proc set 8
OMP: Info #254: KMP_AFFINITY: pid 3649226 tid 3655488 thread 9 bound to OS proc set 9
OMP: Info #254: KMP_AFFINITY: pid 3649226 tid 3655489 thread 10 bound to OS proc set 10
OMP: Info #254: KMP_AFFINITY: pid 3649226 tid 3655490 thread 11 bound to OS proc set 11
OMP: Info #254: KMP_AFFINITY: pid 3649226 tid 3655

Epoch: [ 1], step: [10], time: [1.1690], loss: [0.20777231]
Epoch: [ 1], step: [20], time: [2.1012], loss: [0.37647262]
Epoch: [ 1], step: [30], time: [2.9614], loss: [0.26794398]
Epoch: [ 1], step: [40], time: [3.8149], loss: [0.20024158]
Epoch: [ 1], step: [50], time: [4.7067], loss: [0.18675004]
Epoch: [ 1], step: [60], time: [5.6031], loss: [0.21128026]
Epoch: [ 1], step: [70], time: [6.4764], loss: [0.30240336]
Epoch: [ 1], step: [80], time: [7.3751], loss: [0.22678497]
Epoch: [ 1], step: [90], time: [8.2590], loss: [0.17680673]


KeyboardInterrupt: 